In [1]:
import json
import requests

from time import sleep
from urllib.request import urlretrieve

from bs4 import BeautifulSoup
from IPython.display import display, HTML, Image

In [6]:
def fetch(url):
    return BeautifulSoup(requests.get(url).content, "html.parser")


def get_imgs(soup):
    divs = soup.findAll('div', class_='vendor-rollout-target')

    if not divs:
        return
    
    results = []
    for d in divs:
        img = d.find('img')
        results.append({
            'title': d.find('a').text,
            'alt': img.get('alt'),
            'size': '{}x{}'.format(img.get('width'), img.get('height')),
            'url': img.get('srcset').split(' ')[0],
            'url2': img.get('src'),
        })

    return results

In [3]:
with open('data/popular-emojis.json') as f:
    data = json.load(f)

data[:5]

[[':outbox_tray:', '📤'],
 [':nauseated_face:', '🤢'],
 [':OK_button:', '🆗'],
 [':raising_hands:', '🙌'],
 [':globe_with_meridians:', '🌐']]

In [12]:
img_data = []
for i, d in enumerate(data):
    name, emoji = d
    url = 'http://emojipedia.org/search/?q={}'.format(emoji)
    soup = fetch(url)
    
    try:
        imgs = get_imgs(soup)
    except Exception as e:
        print('uh-oh! error with {}'.format(name))
        continue

    img_data.append({'name': name, 'emoji': emoji, 'imgs': imgs})

    if i % 10 == 0:
        print('{} of {} complete ({})...'.format(i + 1, len(data), name))
        sleep(1)

1 of 363 complete (:outbox_tray:)...
11 of 363 complete (:link:)...
21 of 363 complete (:blossom:)...
31 of 363 complete (:flashlight:)...
41 of 363 complete (:person_getting_massage:)...
51 of 363 complete (:right-pointing_magnifying_glass:)...
61 of 363 complete (:person_pouting:)...
71 of 363 complete (:cloud_with_lightning_and_rain:)...
81 of 363 complete (:volleyball:)...
91 of 363 complete (:lion_face:)...
101 of 363 complete (:candle:)...
111 of 363 complete (:litter_in_bin_sign:)...
121 of 363 complete (:family:)...
131 of 363 complete (:cat_face_with_wry_smile:)...
141 of 363 complete (:call_me_hand:)...
151 of 363 complete (:oncoming_bus:)...
161 of 363 complete (:dove:)...
171 of 363 complete (:shark:)...
181 of 363 complete (:heavy_plus_sign:)...
191 of 363 complete (:nine_o’clock:)...
201 of 363 complete (:four_leaf_clover:)...
211 of 363 complete (:palm_tree:)...
221 of 363 complete (:pensive_face:)...
231 of 363 complete (:tongue:)...
241 of 363 complete (:spoon:)...
251

In [9]:
for i, d in enumerate(img_data):
    name_norm = d['name'].replace(':', '').lower()
    tw, = [i for i in d['imgs'] if i['title'] == 'Twitter']
    urlretrieve(tw['url'], 'img/{}.png'.format(name_norm))

    if i % 5 == 0:
        sleep(1)

In [4]:
stylez = 'margin:0;width:100px;float:left;'
imgs = ''.join([
    "<img style='{}' src='img/{}.png'/>".format(stylez, d[0].replace(':', '').lower())
    for d in data[:20]
])

display(HTML(imgs))